In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
estimators = [
    ('simpleImputer', SimpleImputer()),
    ('pca', PCA()),
    ('regressor', LinearRegression())
]
pipe = Pipeline(steps=estimators)
print(len(pipe.steps))

3


In [ ]:
df = pd.read_csv('/content/GA_2_dataset.csv')
df.head()

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
0,35900,37.0,Male,Other,Strategy,23.929404,NaN,Hard,3,124,99,18,Medium
1,27085,25.0,Male,NaN,Action,22.755168,1.0,Easy,14,84,84,12,Medium
2,39595,24.0,Female,Europe,Simulation,19.505292,0.0,Hard,3,172,9,18,Medium
3,37440,26.0,Female,Europe,RPG,11.009645,NaN,NaN,3,83,36,43,Low
4,22882,17.0,Female,USA,RPG,0.581039,1.0,Medium,5,163,9,24,Medium


In [ ]:
df = df.drop(['PlayerID'], axis=1)
X = df.drop(['EngagementLevel'], axis=1)
y = df['EngagementLevel']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


impute = ColumnTransformer([
    ('Age', SimpleImputer(strategy='mean'), ['Age']),
    ('Location', SimpleImputer(strategy='constant', fill_value='Other'), ['Location']),
    ('GameDifficulty', SimpleImputer(strategy='most_frequent'), ['GameDifficulty']),
    ('InGamePurchases', SimpleImputer(strategy='constant', fill_value=0), ['InGamePurchases'])
], remainder='passthrough',verbose_feature_names_out=False)
impute.set_output(transform='pandas')

fit = impute.fit(X_train)
print(fit)
X_train_imputed = impute.fit_transform(X_train)
print(X_train_imputed)

X_test_imputed = impute.transform(X_test)



ColumnTransformer(remainder='passthrough',
                  transformers=[('Age', SimpleImputer(), ['Age']),
                                ('Location',
                                 SimpleImputer(fill_value='Other',
                                               strategy='constant'),
                                 ['Location']),
                                ('GameDifficulty',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['GameDifficulty']),
                                ('InGamePurchases',
                                 SimpleImputer(fill_value=0,
                                               strategy='constant'),
                                 ['InGamePurchases'])],
                  verbose_feature_names_out=False)
       Age Location GameDifficulty  InGamePurchases  Gender   GameGenre  \
9254  34.0     Asia         Medium              0.0  Female  Simulation   
1561  41.0      USA         Medium         

## Ordinal Encoder

In [ ]:
ordinal_features = ['GameDifficulty']
nominal_features = ['Gender', 'Location', 'GameGenre']
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

ordinal_encoder = OrdinalEncoder(
    categories=[['Easy', 'Medium', 'Hard']]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('ord', ordinal_encoder, ordinal_features),
        ('nom', OneHotEncoder(drop='first'), nominal_features),
        ('num', StandardScaler(), numeric_features)
    ],
    remainder='passthrough'
)

In [ ]:
X_train_transformed = preprocessor.fit_transform(X_train_imputed)
X_test_transformed = preprocessor.transform(X_test_imputed)


first_five = X_test_transformed[:5].sum()
first_five


np.float64(6.83718040298121)